# Imports

In [1]:
import numpy as np
import torch
torch.set_float32_matmul_precision('high')
from train import *

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Experiment settings
Settings are based on the training procedure that produced the final generator used in evaluating the stylized facts and training the RL agent in portfolio management.  
However, using different machines would result in cumulative differences in computation results likely due to varied machine precision.  
Original experiments were performed on machine with following specs:
1. OS: Ubuntu 20.04.1
2. CPU: Intel(R) Xeon(R) Gold 6248 CPU @ 2.50GHz
3. GPU: Tesla V100-SXM2-32GB

### Dataset related

In [2]:
# samples
batch_size = 38 # number of samples in each batch
sample_len = 300 # length of each sample
sample_model = 'Realdt' # GBM, Heston, OU, RealData, Realdt, spx_rates
lead_lag = True # whether to use lead lag transformation
lags = [1] # number of lags to use for lead lag transformation: int or list[int]
seed = 42

# real data parameters
stride = 50 # for real data
start_date = '1995-01-01' # start date for real data
end_date = '2018-09-18' # end date for real data

### Generator and kernel related

In [3]:
# signature kernel
static_kernel_type = 'rq' # type of static kernel to use - rbf, rbfmix, rq, rqmix, rqlinear for
n_levels = 10 # number of levels in the truncated signature kernel

# generator
seq_dim = 1 # dimension of sequence vector
activation = 'Tanh' # pytorch names e.g. Tanh, ReLU. NOTE: does NOT change transformer layers'
hidden_size = 64
n_lstm_layers = 1 # number of LSTM layers
conditional = True # feed in history for LSTM generators
hist_len = 50

### Noise related

In [4]:
noise_dim = 4 # dimension of noise vector
ma = True # whether to use MA noise generator fitted to log returns gaussianized by Lambert W transformation
ma_p = 20

### Training Related

In [5]:
epochs = 10000 # number of batches
start_lr = 0.001 # starting learning rate
patience = 100 # number of epochs to wait before reducing lr
lr_factor = 0.5 # factor to multiply lr by for scheduler
early_stopping = patience*3 # number of epochs to wait before no improvement
kernel_sigma = 0.1 # starting kernel_sigma
num_losses = 20

### Save to tensorboard log

In [6]:
# save all parameters to a dictionary
rng = np.random.default_rng(seed)
torch.manual_seed(seed)

data_params, model_params, train_params = get_params_dicts(vars().copy())

# save parameters to tensorboard
writer = start_writer(data_params, model_params, train_params)

### Data, kernel, generator

In [7]:
dataloader = get_dataloader(**{**data_params, **model_params})
kernel = get_signature_kernel(**{**model_params, **train_params})
generator = get_generator(**{**model_params, **data_params})
generator.to(device)

Optimization terminated successfully    (Exit mode 0)
            Current function value: 5855.988337066719
            Iterations: 36
            Function evaluations: 837
            Gradient evaluations: 36


/home/e0878071/anaconda3/envs/torch/lib/python3.11/site-packages/arch/univariate/base.py:1890: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if isinstance(table_val[pos], (np.float64, float)):
/home/e0878071/anaconda3/envs/torch/lib/python3.11/site-packages/arch/univariate/base.py:1891: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  assert isinstance(table_val[pos], float)
/home/e0878071/anaconda3/envs/torch/lib/python3.11/site-packages/arch/univariate/base.py:1892: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent

                        Zero Mean - ARCH Model Results                        
Dep. Variable:           gaussianized   R-squared:                       0.000
Mean Model:                 Zero Mean   Adj. R-squared:                  0.000
Vol Model:                       ARCH   Log-Likelihood:               -5855.99
Distribution:                  Normal   AIC:                           11754.0
Method:            Maximum Likelihood   BIC:                           11897.9
                                        No. Observations:                 6999
Date:                Tue, Jul 02 2024   Df Residuals:                     6999
Time:                        17:23:03   Df Model:                            0
                               Volatility Model                              
                 coef    std err          t      P>|t|       95.0% Conf. Int.
-----------------------------------------------------------------------------
omega          0.0727  8.049e-03      9.032  1.688e-19 

GenLSTM(
  (rnn): LSTM(6, 64, batch_first=True)
  (mean_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (var_net): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): Tanh()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
  (output_net): Linear(in_features=64, out_features=1, bias=True)
)

# Train MMD-GAN

In [8]:
train(generator, kernel, dataloader, rng, writer, device, **{**train_params, **model_params, **data_params})

100%|██████████| 3/3 [00:05<00:00,  1.85s/it]


Epoch 0, loss: 57737211.333333336, avg_last_20_loss: 57737211.333333336
Saving model at epoch 0


100%|██████████| 3/3 [00:05<00:00,  1.73s/it]


Epoch 1, loss: 580286.2659505209, avg_last_20_loss: 29158748.79964193
Saving model at epoch 1


100%|██████████| 3/3 [00:05<00:00,  1.72s/it]


...

Epoch 4510, loss: -0.9917500813802084, avg_last_20_loss: 0.23923683166503906


100%|██████████| 3/3 [00:05<00:00,  1.71s/it]


Epoch 4511, loss: 2.111577351888021, avg_last_20_loss: 0.4233373006184896


100%|██████████| 3/3 [00:05<00:00,  1.71s/it]

Epoch 4512, loss: 1.005353291829427, avg_last_20_loss: 0.5434401194254558
Early stopping at epoch 4512
